In [2]:
# Importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from unidecode import unidecode

# Carregar dados do CSV em um DataFrame do Pandas
df = pd.read_csv('./dados.csv', delimiter=';')

dataframes_por_letra = {}

# Separe o DataFrame por letra inicial:
for letra in letras:
    dataframes_por_letra[letra] = df[df['Name'].str[0] == letra]

# Obter a contagem de nomes por letra inicial
contagem_por_letra = {}
for letra, dataframe in dataframes_por_letra.items():
    contagem_por_letra[letra] = len(dataframe)

# Calcular a média aproximada de nomes por grupo
media_desejada = np.mean(list(contagem_por_letra.values()))
tamanho_grupo = int(media_desejada)

# Agrupar os nomes em grupos aproximadamente do mesmo tamanho
grupos = []
grupo_atual = []
tamanho_atual = 0

for caixa in caixas:
    grupo_atual = []
    tamanho_atual = 0
    for letra in caixa:
        if letra in dataframes_por_letra:
            dataframe = dataframes_por_letra[letra]
            for _, row in dataframe.iterrows():
                nome = row['Name']
                tamanho_nome = len(nome)
                if tamanho_atual + tamanho_nome <= tamanho_grupo:
                    grupo_atual.append(nome)
                    tamanho_atual += tamanho_nome
                else:
                    grupos.append(grupo_atual)
                    grupo_atual = [nome]
                    tamanho_atual = tamanho_nome

    if grupo_atual:
        grupos.append(grupo_atual)

# Imprimir os grupos de nomes
for i, grupo in enumerate(grupos):
    print(f"Grupo {i+1}: {grupo}")
    print()

    # Divida os dados em recursos e rótulos:
    X = df[df['Name'].isin(grupo)]['Name']
    y = df[df['Name'].isin(grupo)]['Gender']

    # Remova acentos e caracteres problemáticos:
    def remove_accents(text):
        return unidecode(text)

    # Transforme os dados de texto em vetores numéricos usando o CountVectorizer do Scikit-learn:
    vectorizer = CountVectorizer(preprocessor=remove_accents)
    X = vectorizer.fit_transform(X)

    # Divida os dados em conjuntos de treinamento e teste:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Padronize os recursos usando o StandardScaler do Scikit-learn:
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.toarray())
    X_test = scaler.transform(X_test.toarray())

    # Crie um modelo sequencial com o Keras:
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile o modelo com a função de perda, otimizador e métricas:
optimizer = Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

# Treine o modelo com os dados de treinamento:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, epochs=20, batch_size=32,
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Avalie o desempenho do modelo com os dados de teste:
y_pred = model.predict_classes(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia da letra {letra}: ", accuracy)

data.append({'letra': letra, 'acuracia': accuracy})

# Imprimir os resultados
for item in data:
    print(f"Letra: {item['letra']}, Acurácia: {item['acuracia']}")


2023-05-03 21:19:24.440183: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 21:19:24.442030: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 21:19:24.495875: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 21:19:24.496919: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 21:19:25.312469: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

NameError: name 'letras' is not defined